In [1]:
import numpy as np
import os
import sys
import open3d as o3d
import cv2

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from calibration.hand_in_eye import HandinEyeCalibrator
from calibration.utils import read_data

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Calibration

In [2]:
# Read data
base_dir = '../hand_in_eye'
rgb_list, depth_list, pose_list, camera_matrix = read_data(base_dir)
print(f"{len(rgb_list)} poses found")
print(f'Camera matrix: {camera_matrix}')

40 poses found
Camera matrix: [[1.36377332e+03 0.00000000e+00 9.56298462e+02]
 [0.00000000e+00 1.36157141e+03 5.46581238e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [3]:
# Calibrate
charuco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
board = cv2.aruco.CharucoBoard((5, 5), 0.08, 0.06, charuco_dict)

calibrator = HandinEyeCalibrator(camera_matrix, None, charuco_dict, board)
R_cam2gripper_avg, t_cam2gripper_avg = calibrator.perform(rgb_list, pose_list)

print("Average Camera to gripper rotation matrix:")
print(R_cam2gripper_avg)
print("Average Camera to gripper translation vector:")
print(t_cam2gripper_avg)

Average Camera to gripper rotation matrix:
[[ 0.0687324  -0.93363208 -0.35157787]
 [ 0.99431219  0.0928481  -0.05217765]
 [ 0.08135807 -0.34599187  0.93470342]]
Average Camera to gripper translation vector:
[[ 0.1623917 ]
 [ 0.0127938 ]
 [-0.07321238]]


# Visualize results

In [4]:
# visualize coordinates
R_avg = R_cam2gripper_avg
t_avg = t_cam2gripper_avg

R_original = np.eye(3)
t_original = np.zeros((3, 1))

R_relative = R_avg.copy()
t_relative = t_avg.copy()

T_original = np.eye(4)
T_original[:3, :3] = R_original
T_original[:3, 3] = t_original.flatten()

T_relative = np.eye(4)
T_relative[:3, :3] = R_relative
T_relative[:3, 3] = t_relative.flatten()

T_transformed = np.dot(T_original, T_relative)

mesh_frame_original = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6)
mesh_frame_transformed = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6)

mesh_frame_original.transform(T_original)
mesh_frame_transformed.transform(T_transformed)

o3d.visualization.draw_geometries([mesh_frame_original, mesh_frame_transformed])

In [5]:
# visualize pointclouds
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera_intrinsic.set_intrinsics(camera_matrix.shape[1], camera_matrix.shape[0],
                                camera_matrix[0, 0], camera_matrix[1, 1],
                                camera_matrix[0, 2], camera_matrix[1, 2])

# Transformation matrix from camera to gripper
T_cam_to_gripper = np.eye(4)
T_cam_to_gripper[0:3, 0:3] = R_cam2gripper_avg
T_cam_to_gripper[0:3, 3] = t_cam2gripper_avg.flatten()

combined_pcd = o3d.geometry.PointCloud()

# Iterate over all images and add them to the point cloud
for i in range(0,len(rgb_list), 4):
    rgb_img = rgb_list[i]
    depth_img = depth_list[i]
    pose_to_base = pose_list[i]

    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        o3d.geometry.Image(rgb_img),
        o3d.geometry.Image(depth_img),
        depth_scale=1000.0
    )

    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image, camera_intrinsic
    )

    cam_to_world = pose_to_base @ T_cam_to_gripper
    pcd.transform(cam_to_world)

    combined_pcd += pcd

# Visualize the combined point cloud
o3d.visualization.draw_geometries([combined_pcd])